In [ ]:
import fastai
from fastai import *          # Quick access to most common functionality
from fastai.vision import *   # Quick access to computer vision functionality
from fastai.layers import Lambda
from fastai.callbacks import *
import pytorch_ssim as ssim
from superres import *
from torchvision.models import vgg16_bn
import czifile
import shutil
import numbers
from fastai.vision.image import TfmPixel

In [ ]:
#torch.cuda.set_device(3)

In [ ]:
path_data = Path('/DATA/WAMRI/salk/uri')
train_hr = path_data/'Image_restoration_data/train_HR'
train_lr = path_data/'Image_restoration_data/train_HR'
path_test_lr = path_data/'test_imgs'
path_test2_lr = path_data/'newimg'
path_models = Path('/DATA/WAMRI/salk/uri/models')

In [ ]:
def pull_id(fn):
    return fn.split('#')[-1].split('.')[0]

def match_hr_fn(x):
    return hr_names_by_id[pull_id(x.name)]

lr_names_full = list(train_lr.glob('*.tif'))
lr_names_full.sort()
hr_names_by_id = {pull_id(hrfn.name):hrfn for hrfn in train_hr.glob('*.tif')}

In [ ]:
# print(len(fnames))
# for fn in progress_bar(fnames):
#     process_src(fn)

In [ ]:
src = (GrayImageItemList
       .from_folder(train_lr, '*.tif', label_class=GrayImageItemList)
       .split_by_valid_func(lambda x: x.stem[-3] == '0')
       .label_from_func(match_hr_fn))

In [ ]:
def charbonnier_loss(inputs, targets):
    eps = 1e-6;
    d = inputs - targets
    e = torch.sqrt(d**2 + eps)
    return e.mean()


class GaussianSmoothing(nn.Module):
    """
    Apply gaussian smoothing on a
    1d, 2d or 3d tensor. Filtering is performed seperately for each channel
    in the input using a depthwise convolution.
    Arguments:
        channels (int, sequence): Number of channels of the input tensors. Output will
            have this number of channels as well.
        kernel_size (int, sequence): Size of the gaussian kernel.
        sigma (float, sequence): Standard deviation of the gaussian kernel.
        dim (int, optional): The number of dimensions of the data.
            Default value is 2 (spatial).
    """
    def __init__(self, channels, kernel_size, sigma, dim=2):
        super(GaussianSmoothing, self).__init__()
        if isinstance(kernel_size, numbers.Number):
            kernel_size = [kernel_size] * dim
        if isinstance(sigma, numbers.Number):
            sigma = [sigma] * dim

        # The gaussian kernel is the product of the
        # gaussian function of each dimension.
        kernel = 1
        meshgrids = torch.meshgrid(
            [
                torch.arange(size, dtype=torch.float32)
                for size in kernel_size
            ]
        )
        for size, std, mgrid in zip(kernel_size, sigma, meshgrids):
            mean = (size - 1) / 2
            kernel *= 1 / (std * math.sqrt(2 * math.pi)) * \
                      torch.exp(-((mgrid - mean) / (2 * std)) ** 2)

        # Make sure sum of values in gaussian kernel equals 1.
        kernel = kernel / torch.sum(kernel)

        # Reshape to depthwise convolutional weight
        kernel = kernel.view(1, 1, *kernel.size())
        kernel = kernel.repeat(channels, *[1] * (kernel.dim() - 1))

        self.register_buffer('weight', kernel)
        self.groups = channels

        if dim == 1:
            self.conv = F.conv1d
        elif dim == 2:
            self.conv = F.conv2d
        elif dim == 3:
            self.conv = F.conv3d
        else:
            raise RuntimeError(
                'Only 1, 2 and 3 dimensions are supported. Received {}.'.format(dim)
            )

    def forward(self, input):
        """
        Apply gaussian filter to input.
        Arguments:
            input (torch.Tensor): Input to apply gaussian filter on.
        Returns:
            filtered (torch.Tensor): Filtered output.
        """
        return self.conv(input, weight=self.weight, groups=self.groups)
    
    
class Blur(nn.Module):
    def __init__(self):
        super().__init__()
        self.smoothing = GaussianSmoothing(1, 5, 1)
        self.pad = nn.ReflectionPad2d(2)
        
    def forward(self, input):
        return self.smoothing(self.pad(input))

In [ ]:
loss = charbonnier_loss
#loss = F.mse_loss
scale=4
model = DBPNLL(num_channels=1, base_filter=8, feat = 256, num_stages=10, scale_factor=scale)
#model = nn.Sequential(model, Blur())

#model = DBPNLL(num_channels=1, base_filter=16, feat = 256, num_stages=10, scale_factor=scale)
model = nn.DataParallel(model, [0,1,2,3])

In [ ]:
sz_lr = 64
scale,bs = 4,16
sz_hr = sz_lr*scale

learn = build_learner(model, bs, sz_lr, sz_hr, src, loss=loss, model_dir=path_models) #, callback_fns=LossMetrics)
learn.lr_find()
learn.recorder.plot()

In [ ]:
sz_lr = 64
scale,bs = 4,16
sz_hr = sz_lr*scale
epochs, lr = 5, 1e-3
learn = batch_learn(model, bs, sz_lr, sz_hr, lr, epochs, src, 
                    save='combo_1.0_small', 
                    loss=loss, model_dir=path_models)

In [ ]:
sz_lr = 96
scale,bs = 4,16
sz_hr = sz_lr*scale

epochs, lr = 5, 5e-4
learn = batch_learn(model, bs, sz_lr, sz_hr, lr, epochs, src, model_dir=path_models,
                    load='combo_1.0_small', save='combo_1.1_small', loss=loss)

In [ ]:
sz_lr = 128
scale,bs = 4,8
sz_hr = sz_lr*scale

epochs, lr = 5, 2e-4
learn = batch_learn(model, bs, sz_lr, sz_hr, lr, epochs, src, model_dir=path_models,
                    load='combo_1.1_small', save='combo_1.2_small', loss=loss)

In [ ]:
sz_lr = 256
scale,bs = 4,4
sz_hr = sz_lr*scale

epochs, lr = 5, 1e-4
learn = batch_learn(model, bs, sz_lr, sz_hr, lr, epochs, src, model_dir=path_models,
                    load='combo_1.2_small', save='combo_1.3_small', loss=loss)

In [ ]:
def my_pred(self, item:ItemBase, **kwargs):
    self.callbacks.append(RecordOnCPU())
    batch = self.data.one_item(item)
    batch = to_half(batch)
    res = self.pred_batch(batch=batch)
    pred = res[0]
    x = self.callbacks[-1].input
    x = x.float()
    pred = pred.float()
    norm = getattr(self.data,'norm',False)
    if norm:
        x = self.data.denorm(x)
        if norm.keywords.get('do_y',False): pred = self.data.denorm(pred)
    self.callbacks = self.callbacks[:-1]
    ds = self.data.single_ds
    pred = ds.y.analyze_pred(pred, **kwargs)
    out = ds.y.reconstruct(pred, ds.x.reconstruct(x[0])) if has_arg(ds.y.reconstruct, 'x') else ds.y.reconstruct(pred)
    return out, pred, res[0]

In [ ]:
# loss = charbonnier_loss
# scale=4
# n_resblocks=128
# n_feats=32
# res_scale= 0.1
# model = WDSR(scale, n_resblocks, n_feats, res_scale, n_colors_in=1)

test_fns = list(path_test_lr.iterdir())
lil = []
big = []
for fn in test_fns:
    learn = None
    gc.collect()
    img = open_grayscale(fn)
    
    scale,bs = 4,4
    sz_lr = max(img.shape)
    sz_lr = min(int(sz_lr), 400)
    sz_hr = sz_lr*scale
    print(sz_lr, sz_hr)
    data = get_data(src, bs, sz_lr, sz_hr)
    learn = Learner(data, model, loss_func=F.mse_loss, model_dir=path_models).to_fp16().load('combo_1.0_small_best')
    a,b,c = my_pred(learn, img)
    img_big = Image(a.data[0:1,:,:])
    lr_fn = Path(fn.stem + '_LR.png')
    img.save(lr_fn)
    hr_fn = Path(fn.stem + '_HR.png')
    img_big.save(hr_fn)
    lil.append(img)
    big.append(img_big)
    print(lr_fn, hr_fn)

In [ ]:
idx=3
l,b = lil[idx], big[idx]
l

In [ ]:
b